In [0]:
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from pyspark.sql.functions import col

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: mnmcount <file>", file=sys.stderr)
        sys.exit(-1)

Usage: mnmcount <file>


An exception has occurred, use %tb to see the full traceback.

SystemExit: -1


In [0]:
spark = (SparkSession.builder.appName("PythonMnMCount").getOrCreate())

In [0]:
mnm_file=sys.argv[1]
# Leemos como un dataframe
mnm_df2 = (spark.read.format("csv").option("header","true").option("inferSchema","true").csv("/FileStore/shared_uploads/irene.castro@bosonit.com/mnm_dataset.csv"))

In [0]:
mnm_df2.show(n=60,truncate=False)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|TX   |Red   |20   |
|NV   |Blue  |66   |
|CO   |Blue  |79   |
|OR   |Blue  |71   |
|WA   |Yellow|93   |
|WY   |Blue  |16   |
|CA   |Yellow|53   |
|WA   |Green |60   |
|OR   |Green |71   |
|TX   |Green |68   |
|NV   |Green |59   |
|AZ   |Brown |95   |
|WA   |Yellow|20   |
|AZ   |Blue  |75   |
|OR   |Brown |72   |
|NV   |Red   |98   |
|WY   |Orange|45   |
|CO   |Blue  |52   |
|TX   |Brown |94   |
|CO   |Red   |82   |
|CO   |Red   |12   |
|CO   |Red   |17   |
|OR   |Green |16   |
|AZ   |Green |46   |
|NV   |Red   |43   |
|NM   |Yellow|15   |
|WA   |Red   |12   |
|OR   |Green |13   |
|CO   |Blue  |95   |
|WY   |Red   |63   |
|TX   |Orange|63   |
|WY   |Yellow|48   |
|OR   |Green |95   |
|WA   |Red   |75   |
|CO   |Orange|93   |
|NV   |Orange|10   |
|WY   |Green |15   |
|WA   |Green |99   |
|CO   |Blue  |98   |
|CA   |Green |86   |
|UT   |Red   |92   |
|AZ   |Brown |16   |
|CA   |Red   |100  |
|UT   |Red   |77   |
|TX   |Yellow

In [0]:
# Aplicamos la operación agg sum y ordenamos de forma ascendiente.

count_mnm_df=(mnm_df2.select("State","Color","Count").groupBy("State","Color").agg({'Count': 'sum'}).orderBy("sum(Count)",ascending=True))
count_mnm_df.show(n=60,truncate=False)


+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|WY   |Brown |86110     |
|WY   |Yellow|87800     |
|WY   |Orange|87956     |
|OR   |Yellow|88129     |
|UT   |Green |88392     |
|TX   |Blue  |88466     |
|UT   |Brown |88973     |
|CA   |Blue  |89123     |
|OR   |Brown |89136     |
|UT   |Yellow|89264     |
|NV   |Red   |89346     |
|CO   |Red   |89465     |
|OR   |Green |89578     |
|WA   |Blue  |89886     |
|AZ   |Blue  |89971     |
|UT   |Blue  |89977     |
|NV   |Blue  |90003     |
|AZ   |Red   |90042     |
|NM   |Blue  |90150     |
|OR   |Red   |90286     |
|CA   |Orange|90311     |
|OR   |Blue  |90526     |
|TX   |Brown |90736     |
|AZ   |Yellow|90946     |
|CO   |Orange|90971     |
|UT   |Red   |90995     |
|WY   |Blue  |91002     |
|NM   |Green |91160     |
|NM   |Orange|91251     |
|NV   |Green |91331     |
|UT   |Orange|91341     |
|NV   |Yellow|91390     |
|WA   |Orange|91521     |
|CA   |Red   |91527     |
|AZ   |Orange|91684     |
|WY   |Red  

In [0]:
# Aplicamos la función agg avg y ordenamos por estado ascendiente.
count_mnm_df=(mnm_df2.select("State","Color","Count").groupBy("State","Color").agg({'Count': 'avg'}).orderBy("State",ascending=True))
count_mnm_df.show(n=60,truncate=False)

+-----+------+------------------+
|State|Color |avg(Count)        |
+-----+------+------------------+
|AZ   |Blue  |54.99449877750611 |
|AZ   |Orange|54.28300769686205 |
|AZ   |Red   |54.637135922330096|
|AZ   |Brown |54.350412249705535|
|AZ   |Green |54.82219570405728 |
|AZ   |Yellow|54.98548972188634 |
|CA   |Red   |55.26992753623188 |
|CA   |Yellow|55.8693967902601  |
|CA   |Orange|54.502715751357876|
|CA   |Brown |55.740395809080326|
|CA   |Green |54.268717353453276|
|CA   |Blue  |55.59762944479102 |
|CO   |Red   |55.089285714285715|
|CO   |Green |54.71336835960304 |
|CO   |Orange|55.402557856272836|
|CO   |Blue  |55.11032448377581 |
|CO   |Yellow|55.22254503195816 |
|CO   |Brown |56.57729468599034 |
|NM   |Yellow|54.94490521327014 |
|NM   |Orange|54.8054054054054  |
|NM   |Blue  |55.03663003663004 |
|NM   |Red   |56.03491124260355 |
|NM   |Green |54.1973840665874  |
|NM   |Brown |55.392412566686424|
|NV   |Orange|54.865070093457945|
|NV   |Brown |55.81050090525045 |
|NV   |Blue  |

In [0]:
# Filtramos por varios estados y ordenamos por estado.
filmnm_df=(mnm_df2
           .select("State","Color","Count")
           .where((mnm_df2.State == "CA") | (mnm_df2.State == "CO"))
           .groupBy("State","Color").agg(count("Count").alias("Total"))
           .orderBy("State"))

filmnm_df.show()

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|  Blue| 1603|
|   CA|Yellow| 1807|
|   CA|Orange| 1657|
|   CA|   Red| 1656|
|   CA| Brown| 1718|
|   CA| Green| 1723|
|   CO|  Blue| 1695|
|   CO|Yellow| 1721|
|   CO|Orange| 1642|
|   CO| Brown| 1656|
|   CO|   Red| 1624|
|   CO| Green| 1713|
+-----+------+-----+



In [0]:
# Esto no funciona no sé por qué.
conta_mnm_df=(filmnm_df
              .select("State","Color","Total")
              .groupBy("State","Color")
              .agg(col.max("Total"), col.min("Total"), col.avg("Total"))
              .orderBy("State",ascending=True))
conta_mnm_df.show(n=60,truncate=False)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-1733530746910406> in <module>
      3               .select("State","Color","Total")
      4               .groupBy("State","Color")
----> 5               .agg(col.max("Total"), col.min("Total"), col.avg("Total"))
      6               .orderBy("State",ascending=True))
      7 conta_mnm_df.show(n=60,truncate=False)

AttributeError: 'function' object has no attribute 'max'